**Extract co-changes across the commit history**

- co-changes of concepts
- co-changest of config file
- co-changes of config options

**Compute Co-Changed Concepts (Internally)**

In [1]:
import json
import pandas as pd
from itertools import combinations
from collections import Counter
import glob

def summarize_commits(data, window_size):
    """
    Summarize commits in windows of a given size.

    :param data: list of commit data
    :param window_size: size of the commit window
    :return: summarized list of commit data
    """
    summarized_data = []
    for i in range(0, len(data), window_size):
        window = data[i:i+window_size]
        concepts = []
        for commit in window:
            if commit.get('network_data'):
                concepts.extend(commit['network_data'].get('concepts', []))
        summarized_data.append({'concepts': list(set(concepts))})
    return summarized_data


def extract_concept_cochanges(file_path: str, repo_name: str, window_size=1):
    print("Extract concept co-changes from", repo_name, "with window size", window_size)

    # Load the JSON file
    with open(file_path) as file:
        data = json.load(file)

    # Summarize commits in windows of the given size
    summarized_commits = summarize_commits(data.get('config_commit_data', []), window_size)

    # Counter to store co-changes
    concept_counts = Counter()

    # Process each summarized commit
    for commit in summarized_commits:
        concepts = commit.get('concepts', [])
      
        # Generate pairs of technologies and pairs of files
        pairs = [tuple(sorted(pair)) for pair in combinations(concepts, 2)]
        
        # Count each pair
        concept_counts.update(pairs)

    # Prepare DataFrame data
    concept_rows = []
    for (concept1, concept2), count in concept_counts.items():
        concept_rows.append({
            "Co-Changed Concepts": (concept1, concept2),
            "Commit Window": window_size,
            "Concept1": concept1,
            "Concept2": concept2,
            "Changed Internally": count,
            "Percentage Internally": round(count / len(summarized_commits), 2),
        })

    concept_df = pd.DataFrame(concept_rows)

    return concept_df


for file_path in glob.glob(f"../data/analyzed_projects/**"):
    repo_name = file_path.split("/")[-1].split(".")[0]
    dfs = []

    for window_size in [1, 2, 5, 10]:
        df = extract_concept_cochanges(file_path, repo_name, window_size)
        dfs.append(df)

    merged_df = pd.concat(dfs, ignore_index=True)
    merged_df.to_csv(f"../data/concept_cochanges/{repo_name}_concept_cochanges.csv", index=False)

Extract concept co-changes from test-config-repo with window size 1
Extract concept co-changes from test-config-repo with window size 2
Extract concept co-changes from test-config-repo with window size 5
Extract concept co-changes from test-config-repo with window size 10
Extract concept co-changes from pig with window size 1
Extract concept co-changes from pig with window size 2
Extract concept co-changes from pig with window size 5
Extract concept co-changes from pig with window size 10
Extract concept co-changes from music-website with window size 1
Extract concept co-changes from music-website with window size 2
Extract concept co-changes from music-website with window size 5
Extract concept co-changes from music-website with window size 10
Extract concept co-changes from apollo with window size 1
Extract concept co-changes from apollo with window size 2
Extract concept co-changes from apollo with window size 5
Extract concept co-changes from apollo with window size 10
Extract conc

**Compute Co-Changed Concepts (Globally)**

Definitions of columns
- `Across Projects` is an integer, indicating number of projects where the co-changed concepts changed as well at least once
- `Percentage globally` is an float, indicating the percentage of projects where the co-changed concepts changed as well
- `Changed globally` is an integer, indicating the total number of times the co-changed concepts changed in other projects

In [2]:
import os
import pandas as pd


def get_cochanged_concepts(target_df, other_dfs):
    """
    Analyze co-changed concepts in a target file against all other projects to compute global stats.

    :param target_df: dataframe of target project
    :param other_dfs: dataframes of all other projects
    :return target_df: updated dataframe of target project
    """
    # Initialize columns
    target_df['Across Projects'] = 0 # Number of projects where the co-changed concepts changed as well at lea
    target_df['Percentage globally'] = 0 # Percentage of projects where the co-changed concepts changed as well
    target_df['Changed Globally'] = 0 # Totoal number of times the co-changed concepts changed in other projects

    for index, row in target_df.iterrows():
        concepts = row['Co-Changed Concepts']
        commit_window = row['Commit Window']

        for other_df in other_dfs:
            # Find all rows in other_df where the option and commit window match
            matching_rows = other_df[(other_df['Co-Changed Concepts'] == concepts) & (other_df['Commit Window'] == commit_window)]
            match_count = len(matching_rows)

            if match_count > 0:
                # Increment "Across Projects" by 1 (project-level count)
                target_df.loc[index, 'Across Projects'] += 1

                # Increment "Changed globally" by the number of times the concepts changed
                target_df.loc[index, 'Changed Globally'] += match_count

    target_df['Percentage Globally'] = round(target_df['Across Projects'] / len(other_dfs), 2)     

    return target_df

data_dir = "../data/concept_cochanges"

for file_name in glob.glob(data_dir + "/*.csv"):
    target_repo = file_name.split("/")[-1].split("_")[0]
    print("Extract concept co-changed concepts from", target_repo)

    repository_files = [file for file in os.listdir(data_dir) if file.endswith('.csv')]
    repository_dataframes = {file: pd.read_csv(os.path.join(data_dir, file)) for file in repository_files}

    target_file_name = f'{target_repo}_concept_cochanges.csv'
    target_df = repository_dataframes[target_file_name]

    # Use all other files as comparison
    other_dfs = [df for name, df in repository_dataframes.items() if name != target_file_name]

    # Perform the analysis
    updated_target_df = get_cochanged_concepts(target_df.copy(), other_dfs)

    updated_target_df.head(50)

    updated_target_df.to_csv(f"../data/concept_cochanges/{target_repo}_concept_cochanges.csv")

updated_target_df.head(50)

Extract concept co-changed concepts from piggymetrics
Extract concept co-changed concepts from pig
Extract concept co-changed concepts from test-config-repo
Extract concept co-changed concepts from Spring-Cloud-Platform
Extract concept co-changed concepts from apollo
Extract concept co-changed concepts from mall
Extract concept co-changed concepts from mall-swarm
Extract concept co-changed concepts from litemall
Extract concept co-changed concepts from music-website


,Co-Changed Concepts,Commit Window,Concept1,Concept2,Changed Internally,Percentage Internally,Across Projects,Percentage globally,Changed globally
0,"('json', 'nodejs')",1,json,nodejs,158,0.38,2,0.25,2
1,"('configparser', 'spring')",1,configparser,spring,161,0.39,7,0.88,7
2,"('configparser', 'nodejs')",1,configparser,nodejs,161,0.39,2,0.25,2
3,"('configparser', 'maven')",1,configparser,maven,161,0.39,7,0.88,7
4,"('configparser', 'json')",1,configparser,json,156,0.37,6,0.75,6
5,"('nodejs', 'spring')",1,nodejs,spring,161,0.39,2,0.25,2
6,"('maven', 'spring')",1,maven,spring,161,0.39,8,1.00,8
7,"('json', 'spring')",1,json,spring,156,0.37,6,0.75,6
8,"('maven', 'nodejs')",1,maven,nodejs,161,0.39,2,0.25,2
9,"('json', 'maven')",1,json,maven,156,0.37,6,0.75,6


**Compute co-changed config files**

In [3]:
import json
import pandas as pd
from itertools import combinations
from collections import Counter
import glob


def summarize_commits(data, window_size):
    """
    Summarize commits in windows of a given size.

    :param data: list of commit data
    :param window_size: size of the commit window
    :return: summarized list of commit data
    """
    summarized_data = []
    for i in range(0, len(data), window_size):
        window = data[i:i+window_size]
        files = []
        for commit in window:
            if commit.get('network_data'):
                files.extend(commit['network_data'].get('config_files', []))
        summarized_data.append({'config_files': list(set(files))})
    return summarized_data

def extract_file_cochanges(file_path: str, repo_name: str, window_size=1):
    print("Extract concept co-changes from", repo_name, "with window size", window_size)

    # Load the JSON file
    with open(file_path) as file:
        data = json.load(file)

    # Summarize commits in windows of the given size
    summarized_commits = summarize_commits(data.get('config_commit_data', []), window_size)

    # Counter to store co-changes
    file_pair_counts = Counter()

    # Process each summarized commit
    for commit in summarized_commits:
        config_files = commit.get('config_files', [])

        # Generate pairs of files
        file_pairs = [tuple(sorted(pair)) for pair in combinations(config_files, 2)]
        
        # Count each pair
        file_pair_counts.update(file_pairs)

    # Prepare DataFrame data
    file_rows = []
    for (file1, file2), count in file_pair_counts.items():
        file_rows.append({
            "Co-Changed Artifacts": (file1, file2),
            "Commit Window": window_size,
            "Artifact1": file1,
            "Artifact2": file2,
            "Changed Internally": count,
            "Percentage Internally": round(count / len(summarized_commits), 2)
        })

    file_df = pd.DataFrame(file_rows)

    return file_df


for file_path in glob.glob(f"../data/analyzed_projects/**"):
    repo_name = file_path.split("/")[-1].split(".")[0]
    dfs = []

    for window_size in [1, 2, 5, 10]:
        df = extract_file_cochanges(file_path, repo_name, window_size)

        if df.empty:
            print("Empty DataFrame for", repo_name, "with window size", window_size)

        dfs.append(df)

    merged_df = pd.concat(dfs, ignore_index=True)
    merged_df.to_csv(f"../data/file_cochanges/{repo_name}_file_cochanges.csv", index=False)

Extract concept co-changes from test-config-repo with window size 1
Extract concept co-changes from test-config-repo with window size 2
Extract concept co-changes from test-config-repo with window size 5
Extract concept co-changes from test-config-repo with window size 10
Extract concept co-changes from pig with window size 1
Extract concept co-changes from pig with window size 2
Extract concept co-changes from pig with window size 5
Extract concept co-changes from pig with window size 10
Extract concept co-changes from music-website with window size 1
Extract concept co-changes from music-website with window size 2
Extract concept co-changes from music-website with window size 5
Extract concept co-changes from music-website with window size 10
Extract concept co-changes from apollo with window size 1
Extract concept co-changes from apollo with window size 2
Extract concept co-changes from apollo with window size 5
Extract concept co-changes from apollo with window size 10
Extract conc

**Compute Co-Changed Config Files (Globally)**

Definitions of columns
- `Across Projects` is an integer, indicating number of projects where the co-changed artifacts changed as well at least once
- `Percentage globally` is an float, indicating the percentage of projects where the co-changed artifacts changed as well
- `Changed globally` is an integer, indicating the total number of times the co-changed artifacts changed in other projects

In [4]:
import os


def get_cochanged_artifacts(target_df, other_dfs):
    """
    Analyze co-changed artifacts in a target file against all other projects to compute global stats.

    :param target_df: dataframe of target project
    :param other_dfs: dataframes of all other projects
    :return target_df: updated dataframe of target project
    """
    # Initialize columns
    target_df['Across Projects'] = 0
    target_df['Changed Globally'] = 0

    for index, row in target_df.iterrows():
        artifacts = row['Co-Changed Artifacts']
        commit_window = row['Commit Window']

        for other_df in other_dfs:
            # Find all rows in other_df where the option and commit window match
            matching_rows = other_df[(other_df['Co-Changed Artifacts'] == artifacts) & (other_df['Commit Window'] == commit_window)]
            match_count = len(matching_rows)

            if match_count > 0:
                # Increment "Across Projects" by 1 (project-level count)
                target_df.loc[index, 'Across Projects'] += 1

                # Increment "Changed globally" by the number of times the artifacts changed
                target_df.loc[index, 'Changed Globally'] += match_count

    target_df['Percentage Globally'] = round(target_df['Across Projects'] / len(other_dfs), 2)     

    return target_df


data_dir = "../data/file_cochanges"

for file_name in glob.glob(data_dir + "/*.csv"):
    target_repo = file_name.split("/")[-1].split("_")[0]
    print("Extract concept co-changed files from", target_repo)

    repository_files = [file for file in os.listdir(data_dir) if file.endswith('.csv')]
    repository_dataframes = {file: pd.read_csv(os.path.join(data_dir, file)) for file in repository_files}

    target_file_name = f'{target_repo}_file_cochanges.csv'
    target_df = repository_dataframes[target_file_name]

    # Use all other files as comparison
    other_dfs = [df for name, df in repository_dataframes.items() if name != target_file_name]

    # Perform the analysis
    updated_target_df = get_cochanged_artifacts(target_df.copy(), other_dfs)

    updated_target_df.head(50)

    updated_target_df.to_csv(f"../data/file_cochanges/{target_repo}_file_cochanges.csv")

updated_target_df.head(50)

Extract concept co-changed files from mall-swarm
Extract concept co-changed files from piggymetrics
Extract concept co-changed files from pig
Extract concept co-changed files from apollo
Extract concept co-changed files from test-config-repo
Extract concept co-changed files from litemall
Extract concept co-changed files from mall
Extract concept co-changed files from Spring-Cloud-Platform
Extract concept co-changed files from music-website


,Co-Changed Artifacts,Commit Window,Artifact1,Artifact2,Changed Internally,Percentage Internally,Across Projects,Changed globally,Percentage globally
0,"('music-client/package-lock.json', 'music-clie...",1,music-client/package-lock.json,music-client/package.json,81,0.19,0,0,0.0
1,"('music-client/package-lock.json', 'music-serv...",1,music-client/package-lock.json,music-server/pom.xml,79,0.19,0,0,0.0
2,"('music-client/package-lock.json', 'music-serv...",1,music-client/package-lock.json,music-server/.mvn/wrapper/maven-wrapper.proper...,61,0.15,0,0,0.0
3,"('music-client/package-lock.json', 'music-serv...",1,music-client/package-lock.json,music-server/src/main/resources/application.pr...,79,0.19,0,0,0.0
4,"('music-client/package.json', 'music-server/po...",1,music-client/package.json,music-server/pom.xml,161,0.39,0,0,0.0
5,"('music-client/package.json', 'music-server/.m...",1,music-client/package.json,music-server/.mvn/wrapper/maven-wrapper.proper...,143,0.34,0,0,0.0
6,"('music-client/package.json', 'music-server/sr...",1,music-client/package.json,music-server/src/main/resources/application.pr...,161,0.39,0,0,0.0
7,('music-server/.mvn/wrapper/maven-wrapper.prop...,1,music-server/.mvn/wrapper/maven-wrapper.proper...,music-server/pom.xml,143,0.34,0,0,0.0
8,"('music-server/pom.xml', 'music-server/src/mai...",1,music-server/pom.xml,music-server/src/main/resources/application.pr...,161,0.39,0,0,0.0
9,('music-server/.mvn/wrapper/maven-wrapper.prop...,1,music-server/.mvn/wrapper/maven-wrapper.proper...,music-server/src/main/resources/application.pr...,143,0.34,0,0,0.0


**Compute co-changed config options**

In [ ]:
import json
import pandas as pd
from itertools import combinations
from collections import Counter
import glob

def summarize_commits_for_options(data, window_size):
    """
    Summarize commits in windows of a given size for configuration options.

    :param data: list of commit data
    :param window_size: size of the commit window
    :return: summarized list of commits with extracted options
    """
    summarized_data = []
    for i in range(0, len(data), window_size):
        window = data[i:i + window_size]
        modified_options = {}

        for commit in window:
            # Skip if no network data
            if not commit.get('network_data'):
                continue

            # Collect options from all files in this commit
            for file_data in commit['network_data'].get('config_files_data', []):
                for pair in file_data.get('modified_pairs', []):
                    # Store the modified option with its details
                    modified_options[pair['option']] = pair

        # Add unique options from the window to the summary
        summarized_data.append({"modified_options": modified_options})

    return summarized_data


def extract_option_cochanges(file_path: str, repo_name: str, commit_window=1):
    print("Extract option co-changes from", repo_name)

    # Load the JSON file
    with open(file_path) as file:
        data = json.load(file)

    summarized_commits = summarize_commits_for_options(data.get('config_commit_data', []), window_size)

    # Counter to store co-changes
    option_pair_counts = Counter()

    # Process each summarized commit
    for summarized_commit  in summarized_commits:
        modified_options = summarized_commit .get("modified_options", {})
        
        # Generate unique pairs of modified options (by option name)
        option_names = list(modified_options.keys())
        option_pairs = [tuple(sorted(pair)) for pair in combinations(option_names, 2)]

        # Count occurrences of each pair and retain full details
        for option1, option2 in option_pairs:
            pair_details = (
                tuple(sorted(modified_options[option1].items())),
                tuple(sorted(modified_options[option2].items()))
            )
            option_pair_counts[pair_details] += 1

    # Prepare DataFrame data
    rows = []
    for (option1_details, option2_details), count in option_pair_counts.items():
        # Extract details for Option1 and Option2
        opt1 = dict(option1_details)
        opt2 = dict(option2_details)
        rows.append({
            "Co-Changed Options": (opt1.get('option'), opt2.get('option')),
            "Commit Window": window_size,
            "Option1": opt1.get("option"),
            "Values1": (opt1.get("prev_value"), opt1.get("curr_value")),
            "Artifact1": opt1.get("artifact"),
            "Option2": opt2.get("option"),
            "Values2": (opt2.get("prev_value"), opt2.get("curr_value")),
            "Artifact2": opt2.get("artifact"),
            "Changed Internally": count,
            "Percentage Internally": count / len(summarized_commits)
        })

    # Create DataFrame
    option_df = pd.DataFrame(rows)

    option_df.to_csv(f"../data/option_cochanges/{repo_name}_option_cochanges.csv", index=False)

    return option_df


for file_path in glob.glob(f"../data/analyzed_projects/**"):
    repo_name = file_path.split("/")[-1].split(".")[0]
    dfs = []

    for window_size in [1, 2, 5, 10]:
        df = extract_option_cochanges(file_path, repo_name, window_size)
        dfs.append(df)

    merged_df = pd.concat(dfs, ignore_index=True)
    merged_df.to_csv(f"../data/option_cochanges/{repo_name}_option_cochanges.csv", index=False)


Extract option co-changes from test-config-repo
Extract option co-changes from test-config-repo
Extract option co-changes from test-config-repo
Extract option co-changes from test-config-repo
Extract option co-changes from pig


Extract option co-changes from pig
Extract option co-changes from pig
Extract option co-changes from pig
Extract option co-changes from music-website
Extract option co-changes from music-website
Extract option co-changes from music-website
Extract option co-changes from music-website
Extract option co-changes from apollo
Extract option co-changes from apollo
Extract option co-changes from apollo
Extract option co-changes from apollo
Extract option co-changes from mall-swarm
Extract option co-changes from mall-swarm
Extract option co-changes from mall-swarm
Extract option co-changes from mall-swarm
Extract option co-changes from piggymetrics
Extract option co-changes from piggymetrics
Extract option co-changes from piggymetrics
Extract option co-changes from piggymetrics
Extract option co-changes from litemall
Extract option co-changes from litemall
Extract option co-changes from litemall
Extract option co-changes from litemall


**Compute Co-Changed Options (Globally)**

Definitions of columns
- `Across Projects` is an integer, indicating number of projects where the co-changed options changed as well at least once
- `Percentage globally` is an float, indicating the percentage of projects where the co-changed options changed as well
- `Changed globally` is an integer, indicating the total number of times the co-changed options changed in other projects

In [ ]:
import os
import pandas as pd
import glob

def get_cochanged_options(target_df, other_dfs):
    """
    Analyze co-changed options in a target file against all other projects to compute global stats.

    :param target_df: dataframe of target project
    :param other_dfs: dataframes of all other projects
    :return target_df: updated dataframe of target project
    """
    # Initialize columns
    target_df['Across Projects'] = 0
    target_df['Changed Globally'] = 0

    for index, row in target_df.iterrows():
        options = row['Co-Changed Options']

        for other_df in other_dfs:
            # Find all rows in other_df where the option matches
            matching_rows = other_df[other_df['Co-Changed Options'] == options]
            match_count = len(matching_rows)

            if match_count > 0:
                # Increment "Across Projects" by 1 (project-level count)
                target_df.loc[index, 'Across Projects'] += 1

                # Increment "Changed globally" by the total count of matches
                target_df.loc[index, 'Changed Globally'] += match_count

    target_df['Percentage Globally'] = round(target_df['Across Projects'] / len(other_dfs), 2)     

    return target_df

data_dir = "../data/option_cochanges"

for file_name in glob.glob(data_dir + "/*.csv"):
    target_repo = file_name.split("/")[-1].split("_")[0]
    print("Extract concept co-changed files from", target_repo)

    repository_files = [file for file in os.listdir(data_dir) if file.endswith('.csv')]
    repository_dataframes = {file: pd.read_csv(os.path.join(data_dir, file)) for file in repository_files}

    target_file_name = f'{target_repo}_option_cochanges.csv'
    target_df = repository_dataframes[target_file_name]

    # Use all other files as comparison
    other_dfs = [df for name, df in repository_dataframes.items() if name != target_file_name]

    # Perform the analysis
    updated_target_df = get_cochanged_options(target_df.copy(), other_dfs)

    updated_target_df.head(50)

    updated_target_df.to_csv(f"../data/option_cochanges/{target_repo}_option_cochanges.csv")

updated_target_df.head(50)

Extract concept co-changed files from mall
Extract concept co-changed files from mall-swarm
Extract concept co-changed files from music-website
